In [2]:
import collections 
from itertools import chain
import torch 
import time 
import pandas as pd
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import CrossEntropyLoss

from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.evaluation import binary_cls_report, classification_inference
from src.metric import  binary_cls_metrics, binary_cls_log

from models import BertClassifier
from dataset import SeqPairMtlDataset, data_loader
from evaluation import overall_f1
import transformers 
transformers.logging.set_verbosity_error()
from transformers import BertTokenizer,AdamW, get_linear_schedule_with_warmup
device = get_torch_device()

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [3]:
tp = TrainParams(
    log_steps = 10,
    save_steps = 10000,
    epoch_size=20,
    loss_fn=nn.CrossEntropyLoss(),
    max_seq_len=512,
    batch_size=20,
    lr=5e-6,
    weight_decay=0.0,
    epsilon=1e-6,
    warmup_steps=100,
    dropout_rate=0.5,
    label_size=2,
    gradient_clip=1.0,
    hidden_s=200,
    hidden_e=200,
    early_stop_params = {
        'monitor':'f1',
        'mode':'max',
        'min_delta': 0,
        'patience':3,
        'verbose':False
    },
    pretrain_model = 'hfl/chinese-roberta-wwm-ext',
    continue_train=False
)

In [5]:
tokenizer = BertTokenizer.from_pretrained(tp.pretrain_model, do_lower_case=True)
special_tokens_dict = {'additional_special_tokens':['[t]','[c]','[o]','[e]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')

train_dataset = SeqPairMtlDataset(data_loader('/kaggle/input/finneg/trainsample/trainsample/train4.txt'), tp.max_seq_len, tokenizer)
valid_dataset = SeqPairMtlDataset(data_loader('/kaggle/input/finneg/trainsample/trainsample/valid4.txt'), tp.max_seq_len, tokenizer)
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

We have added 4 tokens


In [ ]:
tp.update({'num_train_steps': int(len(train_loader)*tp.epoch_size)})

CKPT = './checkpoint/single_task_bert4'
saver = ModelSave(CKPT, continue_train=False)
saver.init()
es = EarlyStop(**tp.early_stop_params)
global_step = 0
tb = SummaryWriter(CKPT)

model = BertMtl(tp)
model.bert.resize_token_embeddings(len(tokenizer))

model.to(device)
optimizer, scheduler = model.get_optimizer()

In [9]:
for epoch_i in range(tp['epoch_size']):
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
    print("-"*60)

    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    total_loss, batch_loss, batch_counts = 0, 0, 0

    model.train()
    for step, batch in enumerate(train_loader):
        global_step +=1
        batch_counts +=1

        #Forward propogate
        model.zero_grad()
        feature = {k:v.to(device) for k, v in batch.items()}
        logits = model(feature)
        loss = model.compute_loss(feature, logits)
        batch_loss += loss.item()
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), tp.gradient_clip)
        optimizer.step()
        scheduler.step()
        # Log steps for train loss logging
        if (step % tp.log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            time_elapsed = time.time() - t0_batch
            print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
            tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()

        # Save steps for ckpt saving and dev evaluation
        if (step % tp.save_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            val_metrics = binary_cls_metrics(model, valid_loader, device, label_name='label1')
            for key, val in val_metrics.items():
                tb.add_scalar(f'metric/{key}', val, global_step=global_step)
            avg_train_loss = total_loss / step
            tb.add_scalars('loss/train_valid',{'train': avg_train_loss,
                                                'valid': val_metrics['val_loss']}, global_step=global_step)
            saver(total_loss / step, val_metrics['val_loss'], epoch_i, global_step, model, optimizer, scheduler)

    # On Epoch End: calcualte train & valid loss and log overall metrics
    time_elapsed = time.time() - t0_epoch
    val_metrics = binary_cls_metrics(model, valid_loader, device, label_name='label1')
    avg_train_loss = total_loss / step

    print("-"*70)
    print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
    binary_cls_log(epoch_i, val_metrics)
    print("\n")
    if es.check(val_metrics):
        break 

 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   1.468846   |     -     |   12.05  
   1    |   20    |   1.468075   |     -     |   10.14  
   1    |   30    |   1.393609   |     -     |   10.13  
   1    |   40    |   1.369234   |     -     |   10.10  
   1    |   50    |   1.361736   |     -     |   10.11  
   1    |   60    |   1.277964   |     -     |   10.17  
   1    |   70    |   1.310135   |     -     |   10.10  
   1    |   80    |   1.212889   |     -     |   10.13  
   1    |   90    |   1.216510   |     -     |   10.13  
   1    |   100   |   1.070289   |     -     |   10.16  
   1    |   110   |   0.996813   |     -     |   10.12  
   1    |   120   |   0.861399   |     -     |   10.15  
   1    |   130   |   0.790482   |     -     |   10.10  
   1    |   140   |   0.672316   |     -     |   10.11  
   1    |   150   |   0.656919   |     -     |   10.14  
   1    |   160   |   0.5

In [10]:
result = classification_inference(model, valid_loader, device)
valid = pd.read_csv('/kaggle/input/finneg/trainsample/trainsample/valid.csv')
valid['pred'] = result['pred']
valid['prob'] = result['prob']
valid.loc[:,['id','single_entity','pred','prob']].to_csv('valid4.csv')
overall_f1(valid)

{'f1_s': 0.9552058111380144,
 'f1_e': 0.9480381760339343,
 'f1': 0.9509052300755663}